In [12]:
import numpy as np
import random as rnd
import cv2
import os
from glob import glob
import scipy.io

from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import as_completed



In [13]:
def create_dir(output_file):
    if not os.path.exists(output_file):
        os.makedirs(output_file)

In [14]:
def is_inside_polygon(x, y, points):
    """
    Return True if a coordinate (x, y) is inside a polygon defined by
    a list of verticies [(x1, y1), (x2, x2), ... , (xN, yN)].

    Reference: http://www.ariel.com.au/a/python-point-int-poly.html
    """
    n = len(points)
    inside = False
    p1x, p1y = points[0]
    for i in range(1, n + 1):
        p2x, p2y = points[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y
    return inside

In [15]:
def get_counter_points(filename):
    
    mat = scipy.io.loadmat(filename)
    box_coord = mat["box_coord"]
    box_coord = np.reshape(box_coord,[4])
    counter_points = mat["obj_contour"]
    counter_points[0] +=  box_coord[2]
    counter_points[1] +=  box_coord[0]
    counter_points = np.transpose(counter_points).astype(int)
    
    return counter_points


In [16]:
def write_ground_truth(filename,image_height,image_width,counter_points):
    img = np.zeros([image_height,image_width],dtype=np.uint8)
    
    for y in range(image_height):
        for x in range(image_width):
            if is_inside_polygon(x,y,counter_points):
                    img[y][x] = 255
    cv2.imwrite(filename,img)
    

In [25]:

def read_and_write_gt(output_file,img_file,ann_file):
    img = cv2.imread(img_file)
    image_height,image_width,_ = img.shape

    counter_points = get_counter_points(ann_file)
#     cv2.polylines(img,[counter_points],True,(255,0,0),3)
#     cv2.imshow("image",img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    write_ground_truth(output_file,image_height,image_width,counter_points)


In [26]:
IMAGES_BASE_PATH = "101_ObjectCategories/"
ANNOTATIONS_BASE_PATH = "Annotations/"
OUTPUT_BASE_PATH = "101_ObjectCategories_Segmented/"
create_dir(OUTPUT_BASE_PATH)

IS_MULTITHREADED = False
max_workers = 1

imgs_dir_list = os.listdir(IMAGES_BASE_PATH)

if(IS_MULTITHREADED):
    executor = ProcessPoolExecutor(max_workers=max_workers)

futures = []

for imgs_dir in imgs_dir_list:
    img_filenames = sorted(glob(IMAGES_BASE_PATH + imgs_dir +'/*.jpg'))
    ann_filenames = sorted(glob(ANNOTATIONS_BASE_PATH + imgs_dir +'/*.mat'))
    create_dir(OUTPUT_BASE_PATH + imgs_dir)
    for img_file,ann_file in zip(img_filenames,ann_filenames):
        if(len(img_file.split("/")) > 2):
            output_file = OUTPUT_BASE_PATH + img_file.split("/")[1] +"/" + img_file.split("/")[2]
        else:
            output_file = OUTPUT_BASE_PATH + img_file.split("/")[1]
        if(IS_MULTITHREADED):
            futures.append(executor.submit(read_and_write_gt,output_file,img_file,ann_file))
        else:
            read_and_write_gt(output_file,img_file,ann_file)
        break
    break
    #print(imgs_dir + " written")
if(IS_MULTITHREADED):
    for future in as_completed(futures):
        print(future.done(),end=" ")